<div style="text-align: center;">
    <h1 style="
        background: linear-gradient(135deg, #020b22ff, #33485eff);
        color: white;
        padding: 15px 30px;
        border-radius: 500px;
        font-family: 'Segoe UI', Arial, sans-serif;
        box-shadow: 0 4px 15px rgba(0,0,0,0.3);
        display: inline-block;
    ">
       Facial Recognition System using Convolutional Neural Networks
    </h1>
</div>


<h4 style="color:#96c9ffff">1. Importing TensorFlow, OpenCV, Matplotlib, NumPy libraries for data manipulation and image preprocessing with ImageDataGenerator for training preparation.</h4>


In [1]:
import tensorflow as tf
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random
import os
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from pathlib import Path


Dataimages = Path("/content/drive/MyDrive/ML_DEEP/lfw-deepfunneled/lfw")


img_height, img_width = 128, 128
batch_size = 32

validation_split = 0.2


datagen = ImageDataGenerator(rescale=1./255, validation_split=validation_split)


train_generator = datagen.flow_from_directory(
    Dataimages,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)


validation_generator = datagen.flow_from_directory(
    Dataimages,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

print("Data generators created for training and validation.")
print("Number of training batches:", len(train_generator))
print("Number of validation batches:", len(validation_generator))


num_classes = train_generator.num_classes
class_names = list(train_generator.class_indices.keys())

print("Number of classes (individuals):", num_classes)
print("Class names:", class_names[:10])


if validation_generator.samples == 0:
    print("\nWarning: The validation generator is empty. This might be due to the dataset structure (e.g., many classes with very few images) and the validation_split setting.")
    print("Consider adjusting the validation_split ratio or manually splitting your dataset if this issue persists.")

Found 12161 images belonging to 5749 classes.
Found 1072 images belonging to 5749 classes.
Data generators created for training and validation.
Number of training batches: 381
Number of validation batches: 34
Number of classes (individuals): 5749
Class names: ['AJ_Cook', 'AJ_Lamas', 'Aaron_Eckhart', 'Aaron_Guiel', 'Aaron_Patterson', 'Aaron_Peirsol', 'Aaron_Pena', 'Aaron_Sorkin', 'Aaron_Tippin', 'Abba_Eban']


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dense(num_classes, activation='softmax')
])


model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 12, 12, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5749)           │     2,949,237 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,549,877 (21.17 MB)

 Trainable params: 5,549,877 (21.17 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
from tensorflow.keras.optimizers import Adam


model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

print("Model compiled successfully.")

Model compiled successfully.


In [6]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model_checkpoint = ModelCheckpoint('best_facial_recognition_model.keras',
                                   monitor='val_accuracy',
                                   save_best_only=True,
                                   mode='max',
                                   verbose=1)

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=1,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    callbacks=[early_stopping, model_checkpoint]
)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


380/380 ━━━━━━━━━━━━━━━━━━━━ 0s 9s/step - accuracy: 0.0300 - loss: 8.3640
Epoch 1: val_accuracy improved from -inf to 0.09848, saving model to best_facial_recognition_model.keras
380/380 ━━━━━━━━━━━━━━━━━━━━ 3869s 10s/step - accuracy: 0.0300 - loss: 8.3638 - val_accuracy: 0.0985 - val_loss: 6.2474


In [7]:
# Create a test data generator

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    Dataimages,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

print("Test data generator created.")
print("Number of test batches:", len(test_generator))
print("Number of test samples:", test_generator.samples)



Found 13233 images belonging to 5749 classes.
Test data generator created.
Number of test batches: 414
Number of test samples: 13233


In [8]:
# Evaluate the model on the test data
loss, accuracy = model.evaluate(test_generator, steps=len(test_generator))

print(f"\nTest Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

414/414 ━━━━━━━━━━━━━━━━━━━━ 152s 366ms/step - accuracy: 0.0459 - loss: 7.6910

Test Loss: 7.7502
Test Accuracy: 0.0401
